In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [3]:
# Import netcdf-file
filename = '/lustre/storeB/immutable/archive/projects/metproduction/meps/2023/10/10/meps_lagged_6_h_subset_2_5km_20231010T00Z.nc'
data = xr.open_dataset(filename)

In [53]:
# Function for calculating the nearest latitude and longitude coordinates based on pre-defined target values
# 

def find_nearest_coordinates(airport_identifier, target_lat, target_lon):
    latitude_values = data['latitude'].values
    longitude_values = data['longitude'].values

    # Calculate the euclidean distance from the target coordinates
    distances = np.sqrt((latitude_values - target_lat)**2 + (longitude_values - target_lon)**2)

    # Find indices of the minimum distance
    nearest_index = np.unravel_index(distances.argmin(), distances.shape)

    # Extract corresponding y and x indices
    nearest_y_index = nearest_index[0]
    nearest_x_index = nearest_index[1]

    # Extract data using found indices
    print(f'For airport {airport_identifier}: Successfully extracted (y,x) for target_latitude = {target_lat} and target_longitude: {target_lon}')
    return nearest_y_index, nearest_x_index


In [43]:
# Function for locating the value of the corresponding x- and y-values based on index found from find_nearest_coordinates

def extract_vals_to_df(data_param, y_index, x_index):
    
    data_param = data_param.sel(   
        y = data['y'][y_index],   
        x = data['x'][x_index],
        method='nearest'
     )
    data_param = data_param.to_dataframe()
    return data_param

In [12]:
# Test for specific airport
# Target lon and lat
# ENKB: Kristiansund Airport

target_latitude = 63.107166238
target_longitude =  7.822330044 

y_index, x_index = find_nearest_coordinates(target_latitude, target_longitude)
air_temp_2m_ENKB = extract_vals_to_df(data['air_temperature_2m'], y_index, x_index)



Successfully extracted (y,x) for target_latitude = 63.107166238 and target_longitude: 7.822330044


In [13]:
air_temp_2m_ENKB

x          y  \
time                height1 ensemble_member                           
2023-10-10 00:00:00 2.0     0               -360084.0625 -17.904371   
                            1               -360084.0625 -17.904371   
                            2               -360084.0625 -17.904371   
                            3               -360084.0625 -17.904371   
                            4               -360084.0625 -17.904371   
...                                                  ...        ...   
2023-10-12 13:00:00 2.0     25              -360084.0625 -17.904371   
                            26              -360084.0625 -17.904371   
                            27              -360084.0625 -17.904371   
                            28              -360084.0625 -17.904371   
                            29              -360084.0625 -17.904371   

                                             longitude   latitude  \
time                height1 ensemble_member                         
2023-10-10 00:00:00 2.0     0                  7.82299  63.118457   
                            1                  7.82299  63.118457   
                            2                  7.82299  63.118457   
                            3                  7.82299  63.118457   
                            4                  7.82299  63.118457   
...                                                ...        ...   
2023-10-12 13:00:00 2.0     25                 7.82299  63.118457   
                            26                 7.82299  63.118457   
                            27                 7.82299  63.118457   
                            28                 7.82299  63.118457   
                            29                 7.82299  63.118457   

                                             air_temperature_2m  
time                height1 ensemble_member                      
2023-10-10 00:00:00 2.0     0                        281.607910  
                            1                        281.538025  
                            2                        281.012207  
                            3                        280.800049  
                            4                        281.440796  
...                                                         ...  
2023-10-12 13:00:00 2.0     25                       283.175018  
                            26                       281.627747  
                            27                       282.637329  
                            28                       282.832184  
                            29                       283.052490  

[1860 rows x 5 columns]

### Dataset created from extracted positions from postgresql database

In [18]:
airport_location = pd.read_csv('data/airport_positions.csv')
airport_location

,airport_identifier,name,position
0,ENAL,ALESUND/VIGRA RWY 07/25,"(6256,611)"
1,ENAN,ANDOYA/ANDENES RWY 03/21 14/32,"(6929,1614)"
2,ENAS,NY-ALESUND/HAMNERABBEN RWY 12/30,"(7892,1187)"
3,ENAT,ALTA RWY 11/29,"(6997,2337)"
4,ENBJ,BJORNOYA,"(7450,1908)"
...,...,...,...
85,ENVD,VADSO RWY 08/26,"(7006,2984)"
86,ENWV,VALHALL A,"(5627,339)"
87,ENVR,VAEROY,"(6765,1272)"
88,ENXW,GRANE,"(5916,248)"


In [21]:
# Remove parenthesis -> split lon and lat values -> divide by 100 to obtain correct values
airport_location[['latitude', 'longitude']] = airport_location['position'].str.replace(r'[\(\)]', '').str.split(',', expand=True)
airport_location[['latitude', 'longitude']] = airport_location[['latitude', 'longitude']].astype('float') / 100

/tmp/ipykernel_1516454/1650021312.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  airport_location[['latitude', 'longitude']] = airport_location['position'].str.replace(r'[\(\)]', '').str.split(',', expand=True)


In [23]:
airport_location

,airport_identifier,name,position,latitude,longitude
0,ENAL,ALESUND/VIGRA RWY 07/25,"(6256,611)",62.56,6.11
1,ENAN,ANDOYA/ANDENES RWY 03/21 14/32,"(6929,1614)",69.29,16.14
2,ENAS,NY-ALESUND/HAMNERABBEN RWY 12/30,"(7892,1187)",78.92,11.87
3,ENAT,ALTA RWY 11/29,"(6997,2337)",69.97,23.37
4,ENBJ,BJORNOYA,"(7450,1908)",74.50,19.08
...,...,...,...,...,...
85,ENVD,VADSO RWY 08/26,"(7006,2984)",70.06,29.84
86,ENWV,VALHALL A,"(5627,339)",56.27,3.39
87,ENVR,VAEROY,"(6765,1272)",67.65,12.72
88,ENXW,GRANE,"(5916,248)",59.16,2.48


In [50]:
# Function for processing each row from each airport through earlier defined functions

def process_airport_row(row):
    target_lat = row['latitude']
    target_lon = row['longitude']
    airport = row['airport_identifier']

    nearest_y_index, nearest_x_index = find_nearest_coordinates(airport, target_lat, target_lon)
    
    result_df = extract_vals_to_df(data['air_temperature_2m'], nearest_y_index, nearest_x_index)
    
    return result_df

In [51]:
test_dataset = airport_location.head(2)
test_dataset

,airport_identifier,name,position,latitude,longitude
0,ENAL,ALESUND/VIGRA RWY 07/25,"(6256,611)",62.56,6.11
1,ENAN,ANDOYA/ANDENES RWY 03/21 14/32,"(6929,1614)",69.29,16.14


In [54]:
# Applying to dataframe
result = test_dataset.apply(process_airport_row, axis=1)

# Concatenate the results into a final DF
final_result = pd.concat(result.tolist(), keys=test_dataset['airport_identifier'])

final_result

For airport ENAL: Successfully extracted (y,x) for target_latitude = 62.56 and target_longitude: 6.11
For airport ENAN: Successfully extracted (y,x) for target_latitude = 69.29 and target_longitude: 16.14


x  \
airport_identifier time                height1 ensemble_member                  
ENAL               2023-10-10 00:00:00 2.0     0               -455084.062500   
                                               1               -455084.062500   
                                               2               -455084.062500   
                                               3               -455084.062500   
                                               4               -455084.062500   
...                                                                       ...   
ENAN               2023-10-12 13:00:00 2.0     25                44915.945312   
                                               26                44915.945312   
                                               27                44915.945312   
                                               28                44915.945312   
                                               29                44915.945312   

                                                                           y  \
airport_identifier time                height1 ensemble_member                 
ENAL               2023-10-10 00:00:00 2.0     0                -50017.90625   
                                               1                -50017.90625   
                                               2                -50017.90625   
                                               3                -50017.90625   
                                               4                -50017.90625   
...                                                                      ...   
ENAN               2023-10-12 13:00:00 2.0     25               667482.12500   
                                               26               667482.12500   
                                               27               667482.12500   
                                               28               667482.12500   
                                               29               667482.12500   

                                                                longitude  \
airport_identifier time                height1 ensemble_member              
ENAL               2023-10-10 00:00:00 2.0     0                 6.089178   
                                               1                 6.089178   
                                               2                 6.089178   
                                               3                 6.089178   
                                               4                 6.089178   
...                                                                   ...   
ENAN               2023-10-12 13:00:00 2.0     25               16.135429   
                                               26               16.135429   
                                               27               16.135429   
                                               28               16.135429   
                                               29               16.135429   

                                                                 latitude  \
airport_identifier time                height1 ensemble_member              
ENAL               2023-10-10 00:00:00 2.0     0                62.565422   
                                               1                62.565422   
                                               2                62.565422   
                                               3                62.565422   
                                               4                62.565422   
...                                                                   ...   
ENAN               2023-10-12 13:00:00 2.0     25               69.287653   
                                               26               69.287653   
                                               27               69.287653   
                                               28               69.287653   
                                               29      

In [55]:
final_result

x  \
airport_identifier time                height1 ensemble_member                  
ENAL               2023-10-10 00:00:00 2.0     0               -455084.062500   
                                               1               -455084.062500   
                                               2               -455084.062500   
                                               3               -455084.062500   
                                               4               -455084.062500   
...                                                                       ...   
ENAN               2023-10-12 13:00:00 2.0     25                44915.945312   
                                               26                44915.945312   
                                               27                44915.945312   
                                               28                44915.945312   
                                               29                44915.945312   

                                                                           y  \
airport_identifier time                height1 ensemble_member                 
ENAL               2023-10-10 00:00:00 2.0     0                -50017.90625   
                                               1                -50017.90625   
                                               2                -50017.90625   
                                               3                -50017.90625   
                                               4                -50017.90625   
...                                                                      ...   
ENAN               2023-10-12 13:00:00 2.0     25               667482.12500   
                                               26               667482.12500   
                                               27               667482.12500   
                                               28               667482.12500   
                                               29               667482.12500   

                                                                longitude  \
airport_identifier time                height1 ensemble_member              
ENAL               2023-10-10 00:00:00 2.0     0                 6.089178   
                                               1                 6.089178   
                                               2                 6.089178   
                                               3                 6.089178   
                                               4                 6.089178   
...                                                                   ...   
ENAN               2023-10-12 13:00:00 2.0     25               16.135429   
                                               26               16.135429   
                                               27               16.135429   
                                               28               16.135429   
                                               29               16.135429   

                                                                 latitude  \
airport_identifier time                height1 ensemble_member              
ENAL               2023-10-10 00:00:00 2.0     0                62.565422   
                                               1                62.565422   
                                               2                62.565422   
                                               3                62.565422   
                                               4                62.565422   
...                                                                   ...   
ENAN               2023-10-12 13:00:00 2.0     25               69.287653   
                                               26               69.287653   
                                               27               69.287653   
                                               28               69.287653   
                                               29      